In [ ]:
# Kianna's Code

In [ ]:
# Borrowed from Jose
import numpy as np
import pandas as pd
import scipy as sp
import pylab as pl
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib as plt
%matplotlib inline

In [ ]:
# Borrowed from Jose
gene = pd.read_excel('gene_expression_table4.xlsx')
#print(gene)
#datamatrix=[]
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]
print(just_expression)
just_expression.as_matrix  # transformed dataframe with expression values to array
just_expression.shape
dist_matrix= dist.pdist(just_expression)
dist_square_matrix = dist.squareform(dist_matrix)
#print(dist_square_matrix)

In [ ]:
 # gene

In [ ]:
# Structure of this lasso found on :
# https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

# Notes on thie lasso function

#    α = 0: Same coefficients as simple linear regression
#    α = ∞: All coefficients zero
#    0 < α < ∞: coefficients between 0 and that of simple linear regression

#    Goal: 10 coefficients that are not 0

from sklearn.linear_model import Lasso
def lasso_regression(data, predictors, alpha, models_to_plot={}):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(data[predictors],data['y'])
    y_pred = lassoreg.predict(data[predictors])
    
    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(data['x'],y_pred)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for alpha: %.3g'%alpha)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

In [ ]:
#Initialize predictors to all 15 powers of x
predictors=['x']
predictors.extend(['x_%d'%i for i in range(2,16)])

#Define the alpha values to test
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

#Initialize the dataframe to store coefficients
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(0,10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Define the models to plot
models_to_plot = {1e-10:231, 1e-5:232,1e-4:233, 1e-3:234, 1e-2:235, 1:236}

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso.iloc[i,] = lasso_regression(gene, just_expression, alpha_lasso[i], models_to_plot)

In [ ]:
# Jose's
#print(just_expression)
BRCA1_plus= just_expression.iloc[:,0:6]
#print(BRCA1_plus.T)
BRCA1_minus= just_expression.iloc[:,7:12]
#print(BRCA1_minus.T)

BRCA1_plus_means=np.mean(BRCA1_plus.T)
#print(BRCA1_plus_means.shape)
BRCA1_minus_means=np.mean(BRCA1_minus.T)
#print(BRCA1_minus_means.shape)

concatenated_means= np.c_[BRCA1_plus_means,BRCA1_minus_means]
#print(concatenated_means)

expression_change_abs= np.abs(np.diff(concatenated_means))
#print(expression_change_abs.shape)

expression_change_raw= np.diff(concatenated_means)
#print(expression_change_raw)

concatenated_means=np.append(expression_change_abs,expression_change_raw,1) # appended raw and abs value of means
#print(r)

In [ ]:
# Jose's
labels=gene.iloc[0:373,0:3] # isolated gene names and functions
#print(labels.shape)
names=['mean','raw mean']
mean_pandas2= pd.DataFrame(concatenated_means,columns=names) # transformed concatenated means to dataframe
print(mean_pandas2.shape)

mean_append= np.append(labels,mean_pandas2,1) # appended gene labels to concatenated means

names2= ['gene name', 'symbol','function','abs value mean difference','raw mean difference']
labels_and_means =pd.DataFrame(mean_append,columns=names2)

sorted_means=labels_and_means.sort(['abs value mean difference'],ascending=False)
print(sorted_means)

In [ ]:
#from pandas import *
#idx = Int64Index([np.arange(373)])9
#index_sorted_means = DataFrame(index = np.arange(sorted_means.size), data =(sorted_means))
#index_sorted_means
#sorted_means
names_labels=['labels']
index_sorted_means = pd.DataFrame(np.arange(373),columns=names_labels)
#print(index_sorted_means)

data_names= ['labels','gene name', 'symbol','function','abs value mean difference','raw mean difference']
a = np.append(index_sorted_means, sorted_means,1)
my_labels= pd.DataFrame(a, columns=data_names)
my_labels2= my_labels[my_labels.function !='unknown']
my_labels2


In [ ]:
from ggplot import *
limit_graph = my_labels2.iloc[0:20, :]
ggplot(limit_graph, aes(x = 'labels', y = 'abs value mean difference', color = 'function')) + \
    geom_point(size = 45)

In [ ]:
from ggplot import *
ggplot(limit_graph, aes(x = 'labels', y = 'raw mean difference', color = 'function')) + \
    geom_point(size = 45)